# Atividade 3

Desenvolvedores: Cecília e Bruno

Nesta atividade vocês precisarão implementar o modelo gaussiano no Python. Vocês devem realizar os seguintes tópicos:\
-Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas.\
-Implementar a função de estimativa de coeficiente de dispersão (sigmaYZ) para todas as classes de estabilidade.\
-Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs (quem fizer o método de Briggs ganha um ponto a mais). Deve ser considerado o efeito Tip-Downwash\
-Implementar a função do modelo gaussiano\
-Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.\
-Faça figuras e discuta os resultados.


In [54]:
import numpy as np
import pandas as pd

# Definindo a quantidade de valores a serem gerados
quantidade_valores = 20

# Criando uma série de datas para 5 dias no verão e 5 dias no inverno
datas_verao = pd.date_range(start='2023-12-06', periods=10, freq='D')  
datas_inverno = pd.date_range(start='2023-06-21', periods=10, freq='D')  
datas = datas_verao.append(datas_inverno)

# Gerando uma série de cobertura de nuvens (valores entre 0 e 8)
cobertura_nuvens = np.random.randint(0, 9, size=quantidade_valores)

# Gerando uma série de radiação solar usando o método de Monte Carlo com funções seno e cosseno
# Considerando valores entre 0 e 1000 W/m^2
fatores_aleatorios = np.random.rand(quantidade_valores)  # fatores aleatórios entre 0 e 1
radiacao_solar = 500 + 500 * np.sin(2 * np.pi * fatores_aleatorios) * np.cos(2 * np.pi * fatores_aleatorios)

# Gerando uma série de velocidade do vento usando o método de Monte Carlo com funções seno e cosseno
# Considerando valores entre 0 e 15 m/s
velocidade_vento = 7.5 + 7.5 * np.sin(2 * np.pi * fatores_aleatorios) * np.cos(2 * np.pi * fatores_aleatorios)

# Criando um DataFrame para organizar os dados
dados = pd.DataFrame({
    'Data': datas,
    'Cobertura de Nuvens': cobertura_nuvens,
    'Radiação Solar (W/m^2)': radiacao_solar,
    'Velocidade do Vento (m/s)': velocidade_vento
})

# Exibindo o DataFrame
print(dados)

         Data  Cobertura de Nuvens  Radiação Solar (W/m^2)  \
0  2023-12-06                    1              549.660086   
1  2023-12-07                    3              637.627783   
2  2023-12-08                    1              745.537402   
3  2023-12-09                    1              281.954439   
4  2023-12-10                    5              622.743045   
5  2023-12-11                    7              749.220072   
6  2023-12-12                    2              667.291769   
7  2023-12-13                    0              482.157720   
8  2023-12-14                    7              708.665369   
9  2023-12-15                    8              397.658915   
10 2023-06-21                    7              732.992741   
11 2023-06-22                    1              310.950677   
12 2023-06-23                    4              260.431355   
13 2023-06-24                    8              453.855803   
14 2023-06-25                    8              557.800982   
15 2023-

In [55]:
import numpy as np
import pandas as pd

# Função para classificar a estabilidade atmosférica de acordo com o esquema de Pasquil
def classificar_estabilidade(linha):
    velocidade_vento = linha['Velocidade do Vento (m/s)']
    cobertura_nuvens = linha['Cobertura de Nuvens']
    radiacao_solar = linha['Radiação Solar (W/m^2)']
    
    # Classificação baseada na cobertura de nuvens e na velocidade do vento
    if cobertura_nuvens <= 3:  # Céu claro ou poucas nuvens
        if velocidade_vento < 2:
            if radiacao_solar > 600:
                return 'A'  # Muito instável
            elif radiacao_solar > 300:
                return 'B'  # Moderadamente instável
            else:
                return 'C'  # Pouco instável
        elif velocidade_vento < 3:
            if radiacao_solar > 600:
                return 'A'  # Muito instável
            elif radiacao_solar > 300:
                return 'B'  # Moderadamente instável
            else:
                return 'C'  # Pouco instável
        elif velocidade_vento < 5:
            if radiacao_solar > 600:
                return 'B'  # Moderadamente instável
            elif radiacao_solar > 300:
                return 'C'  # Pouco instável
            else:
                return 'D'  # Neutro
        else:
            if radiacao_solar > 600:
                return 'C'  # Pouco instável
            elif radiacao_solar > 300:
                return 'D'  # Neutro
            else:
                return 'D'  # Neutro
    elif cobertura_nuvens <= 6:  # Nublado
        if velocidade_vento < 3:
            return 'D'  # Neutro
        elif velocidade_vento < 5:
            return 'D'  # Neutro
        else:
            return 'D'  # Neutro
    else:  # Céu muito nublado
        if velocidade_vento < 2:
            return 'E'  # Pouco estável
        elif velocidade_vento < 3:
            return 'E'  # Pouco estável
        elif velocidade_vento < 5:
            return 'D'  # Neutro
        else:
            return 'D'  # Neutro

# Gerando os dados fictícios
dados = pd.DataFrame({
    'Data': pd.date_range(start='2024-01-01', periods=20, freq='D'),
    'Cobertura de Nuvens': np.random.randint(0, 9, size=20),
    'Radiação Solar (W/m^2)': 500 + 500 * np.sin(2 * np.pi * np.random.rand(20)) * np.cos(2 * np.pi * np.random.rand(20)),
    'Velocidade do Vento (m/s)': 7.5 + 7.5 * np.sin(2 * np.pi * np.random.rand(20)) * np.cos(2 * np.pi * np.random.rand(20))
})

# Aplicando a função de classificação de estabilidade a cada linha do DataFrame
dados['Classe de Estabilidade'] = dados.apply(classificar_estabilidade, axis=1)

# Exibindo o DataFrame resultante com a nova coluna de classe de estabilidade
print(dados)

         Data  Cobertura de Nuvens  Radiação Solar (W/m^2)  \
0  2024-01-01                    0              965.624090   
1  2024-01-02                    2              424.506271   
2  2024-01-03                    3              521.897574   
3  2024-01-04                    3               16.617570   
4  2024-01-05                    4              397.898617   
5  2024-01-06                    1              851.557629   
6  2024-01-07                    7              853.691113   
7  2024-01-08                    2              336.718264   
8  2024-01-09                    5              791.704421   
9  2024-01-10                    3              501.467681   
10 2024-01-11                    1              261.474797   
11 2024-01-12                    3              644.286773   
12 2024-01-13                    4              286.554506   
13 2024-01-14                    1              410.904227   
14 2024-01-15                    2              947.589645   
15 2024-

In [57]:
import numpy as np

# Função para calcular sigmaY e sigmaZ com base na distância, classe de estabilidade e tipo de ambiente
def calcular_dispersao(distancia, tipo_estabilidade, tipo_ambiente):
    # Verificação do tipo de ambiente
    if tipo_ambiente == 'rural':
        # Cálculos para ambiente rural
        if tipo_estabilidade == 'A':
            dispersao_y = 0.22 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.20 * distancia
        elif tipo_estabilidade == 'B':
            dispersao_y = 0.16 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.12 * distancia
        elif tipo_estabilidade == 'C':
            dispersao_y = 0.11 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.08 * distancia * (1 + 0.0002 * distancia) ** (-0.5)
        elif tipo_estabilidade == 'D':
            dispersao_y = 0.08 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.06 * distancia * (1 + 0.0003 * distancia) ** (-0.5)
        elif tipo_estabilidade == 'E':
            dispersao_y = 0.06 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.03 * distancia * (1 + 0.0003 * distancia) ** (-1)
        elif tipo_estabilidade == 'F':
            dispersao_y = 0.04 * distancia * (1 + 0.0001 * distancia) ** (-0.5)
            dispersao_z = 0.016 * distancia * (1 + 0.0003 * distancia) ** (-1)
        else:
            raise ValueError('Classe de estabilidade inválida')
    elif tipo_ambiente == 'urbano':
        # Cálculos para ambiente urbano
        if tipo_estabilidade in ['A', 'B']:
            dispersao_y = 0.32 * distancia * (1 + 0.0004 * distancia) ** (-0.5)
            dispersao_z = 0.24 * distancia * (1 + 0.001 * distancia) ** 0.5
        elif tipo_estabilidade == 'C':
            dispersao_y = 0.22 * distancia * (1 + 0.0004 * distancia) ** (-0.5)
            dispersao_z = 0.20 * distancia
        elif tipo_estabilidade == 'D':
            dispersao_y = 0.16 * distancia * (1 + 0.0004 * distancia) ** (-0.5)
            dispersao_z = 0.14 * distancia * (1 + 0.0003 * distancia) ** (-0.5)
        elif tipo_estabilidade in ['E', 'F']:
            dispersao_y = 0.11 * distancia * (1 + 0.0004 * distancia) ** (-0.5)
            dispersao_z = 0.08 * distancia * (1 + 0.0015 * distancia) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade inválida')
    else:
        raise ValueError('Tipo de ambiente deve ser "urbano" ou "rural"')

    # Adicionando declarações de impressão para depuração
    print(f"Classe de Estabilidade: {tipo_estabilidade}, Ambiente: {tipo_ambiente}, Distância: {distancia}")
    print(f"Dispersão Horizontal (sigmaY): {dispersao_y}, Dispersão Vertical (sigmaZ): {dispersao_z}")

    return dispersao_y, dispersao_z

# Exemplo de uso da função
distancia = 5000
tipo_estabilidade = 'A'
tipo_ambiente = 'urbano'
dispersao_y, dispersao_z = calcular_dispersao(distancia, tipo_estabilidade, tipo_ambiente)
print(f"Resultado: Dispersão Horizontal (sigmaY) = {dispersao_y}, Dispersão Vertical (sigmaZ) = {dispersao_z}")

Classe de Estabilidade: A, Ambiente: urbano, Distância: 5000
Dispersão Horizontal (sigmaY): 923.7604307034012, Dispersão Vertical (sigmaZ): 2939.3876913398135
Resultado: Dispersão Horizontal (sigmaY) = 923.7604307034012, Dispersão Vertical (sigmaZ) = 2939.3876913398135


In [58]:
import pandas as pd
import numpy as np

# Função para calcular a elevação da pluma de acordo com Briggs, considerando a classe de estabilidade e o efeito Tip-Downwash
def elevacao_pluma(velocidade_vento, altura_chamine, velocidade_saida, diametro_chamine, diferenca_temperatura, temperatura_ambiente, classe_estabilidade):
  
    gravidade = 9.81  # aceleração da gravidade (m/s^2)
    fluxo_flutuabilidade = (gravidade * diametro_chamine**2 * velocidade_saida * diferenca_temperatura) / (4 * temperatura_ambiente)  # fluxo de flutuabilidade
    
    # Calculando a elevação da pluma de acordo com a classe de estabilidade
    if classe_estabilidade == 'C':
        altura_pluma = (2.6 * fluxo_flutuabilidade**0.333 * altura_chamine**0.667) / (velocidade_vento + 0.5 * velocidade_saida)
    elif classe_estabilidade == 'D':
        altura_pluma = (1.6 * fluxo_flutuabilidade**0.333 * altura_chamine**0.667) / (velocidade_vento + 0.5 * velocidade_saida)
    else:
        raise ValueError('Classe de estabilidade inválida. Use "C" ou "D".')
    
    # Considerando o efeito Tip-Downwash
    if velocidade_vento > 1.5 * velocidade_saida:
        altura_pluma *= 0.6  # Reduzindo a elevação da pluma em 40%

    return altura_pluma

# Gerando dados fictícios para exemplo
dados = pd.DataFrame({
    'Velocidade do Vento (m/s)': np.random.uniform(1, 10, 10),
    'Classe de Estabilidade': ['C', 'D', 'C', 'D', 'C', 'D', 'C', 'D', 'C', 'D']
})

# Exemplo de parâmetros variáveis para a chaminé e as condições atmosféricas
altura_chamine = 50  # m
velocidade_saida = 12  # m/s
diametro_chamine = 4  # m
diferenca_temperatura = 100  # K
temperatura_ambiente = 300  # K

# Calculando a elevação da pluma para cada linha do DataFrame usando os parâmetros variáveis
dados['Elevação da Pluma (m)'] = dados.apply(
    lambda linha: elevacao_pluma(
        linha['Velocidade do Vento (m/s)'],
        altura_chamine,
        velocidade_saida,
        diametro_chamine,
        diferenca_temperatura,
        temperatura_ambiente,
        linha['Classe de Estabilidade']
    ), axis=1
)

# Exibindo os dados resultantes
print(dados)

# A condição para reduzir a elevação da pluma em 40% quando a velocidade do vento é significativamente maior (1.5 vezes maior) do que a velocidade de saída dos gases, o que considera o efeito Tip-Downwash. Esta redução é aplicada diretamente ao valor calculado de altura_pluma.

   Velocidade do Vento (m/s) Classe de Estabilidade  Elevação da Pluma (m)
0                   9.189228                      C              12.527038
1                   7.684603                      D               8.556546
2                   2.167524                      C              23.296659
3                   4.518361                      D              11.132242
4                   7.888424                      C              13.700333
5                   2.447347                      D              13.861504
6                   2.694751                      C              21.884012
7                   3.343072                      D              12.532595
8                   7.287361                      C              14.320076
9                   6.584833                      D               9.304290


In [59]:
# Simular para a menor elevação da pluma
menor_elevacao_pluma = dados['Elevação da Pluma (m)'].min()
print(f"Menor valor de elevação da pluma: {menor_elevacao_pluma} m")

Menor valor de elevação da pluma: 8.556546386807366 m


In [72]:
import numpy as np

def modelo_gaussiano(qs, dispersao_y, dispersao_z, velocidade_vento, y, z, altura):
   
    termo1 = qs / (2 * np.pi * dispersao_y * dispersao_z * velocidade_vento)
    termo2 = np.exp((-y**2) / (2 * dispersao_y**2))
    termo3 = np.exp((-(z - altura)**2) / (2 * dispersao_z**2)) + np.exp((-(z + altura)**2) / (2 * dispersao_z**2))
    conc = termo1 * termo2 * termo3
    conc = conc*10**6
    return conc

# Parâmetros fixos e adicionais
qs = 100  # g/s
u = 5  # m/s (velocidade do vento)
y = 0  # Estimativa na direção do vento
z = 1.5  # Altura do nariz
H = 50  # Altura da camada de dispersão

# Chamando a função do modelo gaussiano
conc = modelo_gaussiano(qs, dispersao_y, dispersao_z, u, y, z, H)

# Exibindo os parâmetros e o resultado
print(f'qs: {qs} g/s')
print(f'dispersao_y: {dispersao_y} m')
print(f'dispersao_z: {dispersao_z} m')
print(f'u: {u} m/s')
print(f'y: {y} m')
print(f'z: {z} m')
print(f'H: {H} m')

print(f"\nA concentração estimada é = {conc} g/m³")

qs: 100 g/s
dispersao_y: 923.7604307034012 m
dispersao_z: 2939.3876913398135 m
u: 5 m/s
y: 0 m
z: 1.5 m
H: 50 m

A concentração estimada é = 2.344234256001293 g/m³
